# Code to collect Daily Prices and Sentiment data

## 1. Loading Libraries

In [1]:
import requests
import json
import csv
import pandas as pd
pd.set_option('display.max_columns', None)
from pandas import json_normalize
import glob
import ast 

## 2. Defining API Key and Companies to review

In [5]:
api_key = '6CDMRKZS34KKOJV8'
companies = ['KSCP','ASST','GCT','SNAL','MGAM'] # List of companies should be updated (5 companies per minute) 

## 3. Getting Daily Prices data

In [3]:
duration = 'DAILY'
dailyPricesData = pd.DataFrame()
for ticker in companies: # Change range of values because we can only retrieve data of 5 companies with the free API key
    # Getting data for each ticker
    dailyDataTicker = requests.get(f"https://www.alphavantage.co/query?function=TIME_SERIES_{duration}&symbol={ticker}&apikey={api_key}&outputsize=full").json()
    # Converting data in a Data Frame
    sample = pd.DataFrame(dailyDataTicker['Time Series (Daily)']).transpose().reset_index().rename(columns={'index': 'date'})
    # Adding column with Ticker name
    sample['TickerName'] = ticker
    # Adding Data Frame to monthly data
    dailyPricesData = pd.concat([dailyPricesData, sample], ignore_index=True)

In [10]:
# Visualizing data frame
dailyPricesData

,date,1. open,2. high,3. low,4. close,5. volume,TickerName
0,2023-10-20,1.4000,1.4000,1.2700,1.2700,10514,NA
1,2023-10-19,1.2900,1.3400,1.2900,1.3400,1037,NA
2,2023-10-18,1.2900,1.4000,1.2900,1.4000,22856,NA
3,2023-10-17,1.3362,1.4100,1.3244,1.4000,17842,NA
4,2023-10-16,1.2800,1.4000,1.2300,1.3400,56553,NA
...,...,...,...,...,...,...,...
1341,2022-08-04,6.1000,8.4000,6.0501,6.5500,5153721,MGAM
1342,2022-08-03,7.3800,7.7500,5.5500,6.2100,2612762,MGAM
1343,2022-08-02,10.5100,10.9000,7.3900,8.6300,5646778,MGAM
1344,2022-08-01,16.3000,20.7400,10.2000,12.9800,24345319,MGAM


In [6]:
# Saving data frame as csv file
dailyPricesData.to_csv('newDailyPricesTechCompanies.csv', index=False)

## 4. Getting News & Sentiment data

Topics covered in this script: ipo, technology, and earnings. We need to run the same code for each topic. More topics could be add if required. See: https://www.alphavantage.co/documentation/#news-sentiment

Sentiment Score definition:

x <= -0.35          : Bearish

-0.35 < x <= -0.15  : Somewhat-Bearish

-0.15 < x < 0.15    : Neutral

0.15 <= x < 0.35    : Somewhat_Bullish

x >= 0.35           : Bullish'

In [7]:
# Creating empty List
sentimentData = list()
# Getting News & Sentiment for compnaies
for ticker in companies:
    # Getting overview of each compnay
    #url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={ticker}&apikey={api_key}&limit=1000&sort=LATEST&topics=ipo'
    #url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={ticker}&apikey={api_key}&limit=1000&sort=LATEST&topics=technology'
    url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={ticker}&apikey={api_key}&limit=1000&sort=LATEST&topics=earnings'
    urlContent = requests.get(url).json()
    # Adding dictionary to list
    sentimentData.append(urlContent)
# Creating data frame with list
sentimentNews = pd.DataFrame(sentimentData)
# Removing companies without articles
sentimentNews = sentimentNews[sentimentNews['items'] != '0']
# For each line, feed has a list of dictionaries.
# Each dicitonary represents an article. We will create a row per article
sentimentNews = sentimentNews.explode('feed')
sentimentNews.reset_index(drop=True,inplace=True)
# The key of each dictionary describes a feature. We will create a new column for each dictionary key
feedData = pd.json_normalize(sentimentNews['feed'])
sentimentNews = pd.concat([sentimentNews,feedData], axis=1).drop(['feed','items',
                                                                  'sentiment_score_definition',
                                                                  'relevance_score_definition'], axis=1)
# Even though the API provides us with an overall sentiment score and label columns,
# column ticker_sentiment contains a list of dictionaries with sentiment scores and labels per company.
# First, we will create a column to record the number of companies related to the article
sentimentNews['numberOfCompanies'] = sentimentNews.apply(lambda row: len(row['ticker_sentiment']), axis=1)
# Creating a row per company (Dictionary)
sentimentNews = sentimentNews.explode('ticker_sentiment')
sentimentNews.reset_index(drop=True,inplace=True)
# Expanding keys of dictionaries
tickerSentimentData = pd.json_normalize(sentimentNews['ticker_sentiment'])
sentimentNews = pd.concat([sentimentNews,tickerSentimentData], axis=1).drop(['ticker_sentiment'], axis=1)
sentimentNews = sentimentNews.rename(columns={'ticker':'companyName',
                                              'relevance_score':'companyRelevanceScore',
                                              'ticker_sentiment_score':'companySentimentScore',
                                              'ticker_sentiment_label':'companySentimentLabel'})
# Each article is related to different topics
# Let's create a row per topic
sentimentNews = sentimentNews.explode('topics')
sentimentNews.reset_index(drop=True,inplace=True)
# Let's create a column for the topic and topic relevance score
topicData = pd.json_normalize(sentimentNews['topics'])
sentimentNews = pd.concat([sentimentNews,topicData], axis=1).drop(['topics','authors'], axis=1)
sentimentNews = sentimentNews.rename(columns={'relevance_score':'topicRelevanceScore'})
# Since more than one company can be related to the article, let's remove duplicate rows
sentimentNews = sentimentNews.drop_duplicates()
sentimentNews.reset_index(drop=True,inplace=True) 

In [13]:
sentimentNews

,title,url,time_published,summary,banner_image,source,category_within_source,source_domain,overall_sentiment_score,overall_sentiment_label,numberOfCompanies,companyName,companyRelevanceScore,companySentimentScore,companySentimentLabel,topic,topicRelevanceScore
0,Nano Labs Announces First Half of 2023 Financi...,https://www.globenewswire.com/news-release/202...,20230815T113000,"HANGZHOU, China, Aug. 15, 2023 ( GLOBE NEWSWIR...",https://ml.globenewswire.com/Resource/Download...,GlobeNewswire,n/a,www.globenewswire.com,0.164393,Somewhat-Bullish,5,BACHY,0.022707,0.063029,Neutral,Earnings,1.0
1,Nano Labs Announces First Half of 2023 Financi...,https://www.globenewswire.com/news-release/202...,20230815T113000,"HANGZHOU, China, Aug. 15, 2023 ( GLOBE NEWSWIR...",https://ml.globenewswire.com/Resource/Download...,GlobeNewswire,n/a,www.globenewswire.com,0.164393,Somewhat-Bullish,5,BACHY,0.022707,0.063029,Neutral,Blockchain,0.576289
2,Nano Labs Announces First Half of 2023 Financi...,https://www.globenewswire.com/news-release/202...,20230815T113000,"HANGZHOU, China, Aug. 15, 2023 ( GLOBE NEWSWIR...",https://ml.globenewswire.com/Resource/Download...,GlobeNewswire,n/a,www.globenewswire.com,0.164393,Somewhat-Bullish,5,BACHY,0.022707,0.063029,Neutral,Financial Markets,0.614606
3,Nano Labs Announces First Half of 2023 Financi...,https://www.globenewswire.com/news-release/202...,20230815T113000,"HANGZHOU, China, Aug. 15, 2023 ( GLOBE NEWSWIR...",https://ml.globenewswire.com/Resource/Download...,GlobeNewswire,n/a,www.globenewswire.com,0.164393,Somewhat-Bullish,5,CURN,0.022707,0.03891,Neutral,Earnings,1.0
4,Nano Labs Announces First Half of 2023 Financi...,https://www.globenewswire.com/news-release/202...,20230815T113000,"HANGZHOU, China, Aug. 15, 2023 ( GLOBE NEWSWIR...",https://ml.globenewswire.com/Resource/Download...,GlobeNewswire,n/a,www.globenewswire.com,0.164393,Somewhat-Bullish,5,CURN,0.022707,0.03891,Neutral,Blockchain,0.576289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12117,"( ABSI ) , ( AHI ) - Why Baudax Bio Is Trad...",https://www.benzinga.com/news/22/08/28681565/w...,20220830T105444,Mobile Global Esports Inc. MGAM shares rose 28...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Trading,www.benzinga.com,0.104985,Neutral,29,TAL,0.117774,0.0,Neutral,Retail & Wholesale,0.166667
12118,"( ABSI ) , ( AHI ) - Why Baudax Bio Is Trad...",https://www.benzinga.com/news/22/08/28681565/w...,20220830T105444,Mobile Global Esports Inc. MGAM shares rose 28...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Trading,www.benzinga.com,0.104985,Neutral,29,TAL,0.117774,0.0,Neutral,Financial Markets,1.0
12119,"( ABSI ) , ( AHI ) - Why Baudax Bio Is Trad...",https://www.benzinga.com/news/22/08/28681565/w...,20220830T105444,Mobile Global Esports Inc. MGAM shares rose 28...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Trading,www.benzinga.com,0.104985,Neutral,29,TAL,0.117774,0.0,Neutral,Manufacturing,0.166667
12120,"( ABSI ) , ( AHI ) - Why Baudax Bio Is Trad...",https://www.benzinga.com/news/22/08/28681565/w...,20220830T105444,Mobile Global Esports Inc. MGAM shares rose 28...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Trading,www.benzinga.com,0.104985,Neutral,29,TAL,0.117774,0.0,Neutral,Earnings,0.310843


In [8]:
# Saving data frame as csv file
sentimentNews.to_csv('SentimentNewsTechCompanies.csv', index=False)